###패키지 import & data 불러오기

In [1]:
### colab 사용 시 실행
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
import os
gdrive_path = '/content/gdrive/My Drive/Google Colaboratory/kakao_arena/kakao_arena_melon/'
os.chdir(gdrive_path)   # google drive 내 위치

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
from src.basic_utils import *
from src.modeling_utils import *

from sklearn.metrics.pairwise import cosine_similarity
# from scipy.sparse.linalg import svds
# import pickle #psutil, 

In [3]:
data_path = 'data/'

train = pd.read_json(data_path+'train.json')
val = pd.read_json(data_path+'val.json')
test = pd.read_json(data_path+'test.json')
song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
tag_song_dict = load_json(data_path+'tag_song_dict.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

### data preprocessing

In [4]:
## year 생성 (실제 년도 - 4)
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]) - 4)
year_list = sorted(list(train['year'].unique()))

## 연도별 most popular 노래 및 태그 dict 생성
popular_year = []
for i in year_list:
    tmp = train[train['year'] == i]
    _, year_song = most_popular(tmp, "songs", 200)
    _, year_tag = most_popular(tmp, "tags", 100)
    popular_year.append({
        "year" : i,
        "songs" : year_song,
        "tags" : year_tag,})

In [11]:
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )    # 연도+월
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))                                    # 연도
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))
test['date'] = test['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
test['year'] = test['updt_date'].apply(lambda x: int(x[2:4]))

date_list = list(train['date'].unique())
date_list= sorted(date_list)

date_dict = {}       # 해당 연도+월 에 index 부여(?)
for i,k in enumerate(date_list):
    date_dict[k] = i

def make_popular_date_dict(var, data, date_list, thres):
    popular_date_dict = []
    for i in range(len(date_list)):
        if i <= thres:
            year = int(str(date_list[i])[0])
            tmp = data[data['year'] == year]
        else:
            tmp = data[data['date'] == date_list[i]]
        _, most_var = most_popular(tmp, var, 100)
        popular_date_dict.append({
            "date" : date_list[i],
            var : most_var,
        })
    return popular_date_dict

popular_date_tag = make_popular_date_dict('tags', train, date_list, 13)
popular_date_song = make_popular_date_dict('songs', train, date_list, 9)

In [5]:
# train 내 popular 태그 및 노래 counter, list 추출
tag_mp_counter, tag_mp = most_popular(train, "tags", 100)
song_mp_counter, song_mp = most_popular(train, "songs", 200)

In [6]:
# song_meta를 노래별 dict 형식으로 변환
song_meta_dict = {int(song["id"]): song for song in song_meta}

In [ ]:
## 장르 소분류별 노래 id dict
song_mp_per_genre_dtl = {}
for sid, song in song_meta_dict.items():
    for genres in song['song_gn_dtl_gnr_basket']:
        for genre in [genres]:
            song_mp_per_genre_dtl.setdefault(genre, []).append(sid)
        ## 특정 장르에 해당하는 모든 음악을 담은 데이터 => res

## 장르 대분류별 노래 id dict
song_mp_per_genre = {}
for sid, song in song_meta_dict.items():
    for genre in song['song_gn_gnr_basket']:
        song_mp_per_genre.setdefault(genre, []).append(sid)
        ## 특정 장르에 해당하는 모든 음악을 담은 데이터 => res

In [ ]:
for genre, sids in song_mp_per_genre_dtl.items():
    ## 각 장르에 해당하는 노래 중에서 가장 많이 반복되는 상위 200개 노래들만 남김 
    song_mp_per_genre_dtl[genre] = Counter({k: song_mp_counter.get(int(k), 0) for k in sids})
    song_mp_per_genre_dtl[genre] = [k for k, v in song_mp_per_genre_dtl[genre].most_common(200)]
    if len(song_mp_per_genre_dtl[genre]) < 200:
        ## 200개 미만일 경우 장르 대분류에
        add = 300-len(song_mp_per_genre_dtl[genre])
        big_genre = genre[:4] + '00'
        song_mp_per_genre_dtl[genre].extend([k for k, v in Counter(song_mp_per_genre[big_genre]).most_common(add)]) ## 부족한 거 채워주고

### predict validation

In [12]:
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]) - 4)
test['year'] = test['updt_date'].apply(lambda x: int(x[2:4]) - 4)

no_tag, no_song, yes_index, no_both = check_target_type(val)
val1 = val[val.index.isin(no_tag)]

no_tag, no_song, yes_index, no_both = check_target_type(test)
test1 = test[test.index.isin(no_tag)]

노래 O 태그 X : 9661개
노래 X 태그 O : 2630개
노래 O 태그 O : 8975개
노래 X 태그 X : 1749개
노래 O 태그 X : 4507개
노래 X 태그 O : 1232개
노래 O 태그 O : 4190개
노래 X 태그 X : 811개


In [15]:
def add_var1(date, var1, var2_list, var2_var1_dict, popular_date_var1, cur_var1):
    var1_counter = Counter()
    for v2 in var2_list:
        if str(v2) in var2_var1_dict:
            for v1 in var2_var1_dict[str(v2)]:
                var1_counter.update({var1 : 1})
    var1_counter = sorted(var1_counter.items(), key=lambda x : x[1], reverse=True)
    for k in var1_counter[:100]:
        if k[0] not in cur_var1:
            cur_var1.append(k[0])
    if len(cur_var1) == 0:
        cur_var1 = popular_date_var1[date][var1][:100]
    elif len(cur_var1) < 100:
        update_var1 = remove_seen(cur_var1, popular_date_var1[date][var1])
        update_var1 = [u for u in update_var1 if u not in cur_var1]
        cur_var1.extend(update_var1)
    return cur_var1

In [18]:
def pred_v1(data):
    pred_list = []
    for i in tqdm(data.index):
        date = date_dict[data.loc[i]['date']]
        cur_tag = []
        cur_tag = add_var1(date, 'tags', data.loc[i]['songs'], song_tag_dict, popular_date_tag, cur_tag)[:10]
        cur_song = []
        cur_song = add_var1(date, 'songs', cur_tag[:10], tag_song_dict, popular_date_song, cur_song)
        cur_song = remove_seen(data.loc[i]['songs'], cur_song)[:100]
        pred_list.append({
            "id" : data.loc[i]['id'],
            "songs": cur_song,
            "tags": cur_tag,
        })
    return pred_list

In [ ]:
val1_predict = pred_v1(val1)

In [ ]:
test1_predict = pred_v1(test1)

#### val1 : 태그가 없는 케이스

In [ ]:
val1 = val[val.index.isin(no_tag)]
# val1.reset_index(inplace = True, drop = True)
val1.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,year
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,15
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,4
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,13
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,13
6,[],127575,,"[227044, 424672, 515592, 22408, 57893, 153714,...",2,2011-10-28 11:21:51.000,7


In [ ]:
##### 연도별 나눠서 자동으로 되도록 재구성 필요
### ex : 16년도 예측
train_years =  [i for i in range(0,17)]
small_years =  [16]

In [ ]:
val_tmp = val1[val1['year'].isin(small_years)]
# val_tmp.reset_index(inplace = True, drop = True)
print("val year shape", val_tmp.shape)

# print("train year", train[(train['year'].isin(train_years))].shape)
tmp = train[(train['year'].isin(train_years))]
like_num = int(np.percentile(tmp['like_cnt'], 95))

tmp = tmp[(tmp['like_cnt'] > like_num)]
# tmp.reset_index(inplace = True, drop = True)
print("train best playlist", tmp.shape)
print('train+valid=', tmp.shape[0]+val_tmp.shape[0])

val year shape (1014, 7)
train best playlist (5743, 7)
train+valid= 6757


In [ ]:
def before_pivot(data):
    # id별 포함된 노래 id dataframe 생성
    ## 속도가 느리므로 개선 필요해보임
    result = []
    for i in tqdm(data.index):
        for song in data.loc[i]['songs']:
            df = pd.DataFrame({
                        'id' : [data.loc[i]['id']],
                        'song' : [song],
                        'point' : 1
                    })
            result += [df]
    final_result = pd.concat(result)
    print(final_result.shape)
    return final_result

before_tmp = before_pivot(tmp)

100%|██████████| 5743/5743 [07:15<00:00, 13.18it/s]


(439729, 3)


In [ ]:
## train만 1보다 작은 노래는 없애준다.
tmp_dict = before_tmp['song'].value_counts().to_dict()
before_tmp['count']= before_tmp['song'].apply(lambda x : tmp_dict[x])
before_tmp = before_tmp[before_tmp['count'] > 1]
print(before_tmp.shape)

(343550, 4)


In [ ]:
before_tmp.head()

,id,song,point,count
0,69252,418694,1,27
0,69252,222305,1,49
0,69252,96545,1,14
0,69252,135950,1,32
0,69252,304687,1,9


In [ ]:
before_val_tmp = before_pivot(val_tmp)

(32188, 3)


In [ ]:
final_result = pd.concat([before_tmp, before_val_tmp])
del before_tmp, before_val_tmp
gc.collect()

0

In [ ]:
### colab에서 RAM 초과 오류 발생
def make_cosine_predict(final_result):
    R_df = final_result.pivot(index = 'id', columns ='song', values = 'point').fillna(0)
    del final_result
    
    ##### 코사인 유사도 계산
    print("make cosine data")
    cosine_array = cosine_similarity(R_df, R_df)
    predicted_array = np.zeros(shape=(len(R_df.index), len(R_df.columns))) 
    for i in tqdm(range(len(cosine_array))):
        top_200 = cosine_array[i].argsort()[-201:][::-1]
        top_200 = np.delete(top_200, 0)
        weighted_sum = np.array([0])
        for top_idx in top_200:
            weighted_sum = weighted_sum + (cosine_array[i][top_idx] * R_df.values[top_idx])
        predicted = weighted_sum / len(top_200)
        predicted_array[i] = predicted
        gc.collect()
    iu_predicted = R_df.values*(-99999) + predicted_array     # 기존 pivot table에 predicted array 더한 결과
    
    ##### 플레이리스트 id별 코사인 유사도 높은 상위 200곡 dict 생성
    print("make dic data")
    cf_dic = {}
    for i in range(len(iu_predicted)):
        cf_dic[R_df.index[i]] = R_df.columns[iu_predicted[i].argsort()[-200:][::-1]].tolist()    
        gc.collect()
    return cf_dic

cf_dic = make_cosine_predict(final_result)

In [ ]:
val1_predict = []
for i in tqdm(val_tmp.index):
    ##### song 채우기
    cur_song = add_song_by_cf_dic(val_tmp.loc[i], cf_dic)
    # cur_song = add_song(val_tmp, i)
    ##### tag 채우기
    cur_tag = add_var(val_tmp.loc[i], 'tags', 'songs', song_tag_dict, popular_year, data.loc[i]['year'])
    # cur_tag = add_tag(val_tmp, i, song_tag_dict, popular_year)
    ### val1_predict(제출 결과물)에 추가
    val1_predict.append({
        "id" : val_tmp.loc[i]['id'],
        "songs": cur_song,
        "tags": cur_tag,
    })

In [ ]:
## save file
write_json(val1_predict, data_path+'val1_16.json')

---

In [ ]:
def add_song(data, i):
    row_num = data.loc[i]['id']
    cur_song = remove_seen(data.loc[i]['songs'], cf_dic[row_num])[:100]
    return cur_song

def add_tag(data, i, song_tag_dict, popular_year):
    year = data.loc[i]['year']
    tag_counter = Counter()
    for song in data.loc[i]['songs']:
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key=lambda x : x[1], reverse=True)
    cur_tag = []
    for k in tag_counter[:10]:
        cur_tag.append(k[0])
    if len(cur_tag) == 0:
        cur_tag = popular_year[year]['tags'][:10]
    elif len(cur_tag) < 10:
        update_tag = remove_seen(cur_tag, popular_year[year]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:10]
    return cur_tag

In [ ]:
val1_predict = []
for i in tqdm(val_tmp.index):
    ##### song 채우기
    year = val_tmp.loc[i]['year']
    row_number = val_tmp.loc[i]['id']
    cur_song = remove_seen(val_tmp.loc[i]['songs'], list(cf_dic[row_number]))[:100]
    ##### tag 채우기
    tag_counter = Counter()
    for song in val_tmp.loc[i]['songs']:
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    cur_tag = []
    for k in tag_counter[:10]:
        cur_tag.append(k[0])     
    if len(cur_tag) == 0:
        cur_tag = popular_year[year]['tags'][:10]
    elif len(cur_tag) < 10:
        update_tag = remove_seen(cur_tag, popular_year[year]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:10]
    ### val1_predict(제출 결과물)에 추가
    val1_predict.append({
        "id" : val_tmp.loc[i]['id'],
        "songs": cur_song,
        "tags": cur_tag,
    })